<a href="https://colab.research.google.com/github/Jojocko/Feedback-users-supply-chain/blob/main/M%C3%A9thode_2_multiclustering_apr%C3%A8s_sbert_finetuned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

df = pd.read_csv('/content/drive/My Drive/Datasets NLP/dataset_supplychain.csv')

Mounted at /content/drive


In [2]:
df = df.drop(['client', 'langage', 'reponse'], axis=1)
df.rename(columns={'Sentiment': 'sentiment', 'Commentaire': 'commentaire'}, inplace=True)
df['sentiment'] = df['sentiment'].replace({'__label__POSITIVE': 'positif', '__label__NEGATIVE': 'negatif', '__label__NEUTRAL': 'neutre'})
df['date'] = df['date'].fillna(method="ffill")
df['date'] = pd.to_datetime(df['date'])


In [3]:
df_paires = pd.read_csv('/content/drive/My Drive/Datasets NLP/paires_commentaires.csv')
df_paires.head()

,Commentaire A,Commentaire B,Distance sémantique,Themes
0,"""Prévu pour une livraison express, mais ça a p...","""Impossible d'obtenir une réponse claire sur m...",0.1,retards de livraison
1,"""Commandé bien avant ma randonnée, mais les bo...","""Après avoir expliqué mon problème, le service...",0.1,retards de livraison
2,"""Le retard de livraison a gâché mon projet de ...","""Plus de 30 minutes d'attente pour parler à qu...",0.1,retards de livraison
3,"""Ma commande de Noël est arrivée en janvier......","""Le service client m'a donné des informations ...",0.1,retards de livraison
4,"""J'ai pris un jour de congé pour attendre mon ...","""Promesse de rappel jamais tenue par le servic...",0.1,retards de livraison


In [4]:
# Traitement des commentaires
import re
import nltk
import unicodedata
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')

stop_words = set(stopwords.words('french'))
company = {'Fnac', 'Amazon', 'CDiscount'}
stop_words.update(company)

def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def contractions(text):
    text = text.replace("c'est", "ce est").replace("j'ai", "je ai")
    return text

def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())
    w = contractions(w)
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    w = re.sub(r"[^a-zàâçéèêëîïôûùüÿñæœ?.!,¿]+", " ", w)

    mots = word_tokenize(w.strip())
    mots = [mot for mot in mots if mot not in stop_words]
    return ' '.join(mots).strip()

df['cleaned_lemma'] = df['cleaned_lemma'].apply(preprocess_sentence)
df_paires['Commentaire A'] = df_paires['Commentaire A'].apply(preprocess_sentence)
df_paires['Commentaire B'] = df_paires['Commentaire B'].apply(preprocess_sentence)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [5]:
# Finetuning du transformer

!pip install sentence-transformers
from sentence_transformers import SentenceTransformer
from sentence_transformers import InputExample, losses
from torch.utils.data import DataLoader
from torch.optim import AdamW

model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

paires = [InputExample(texts=[row['Commentaire A'], row['Commentaire B']], label=float(row['Distance sémantique'])) for index, row in df_paires.iterrows()]
train_set = DataLoader(paires, shuffle=True, batch_size=500)
train_loss = losses.CosineSimilarityLoss(model=model)

model.fit(train_objectives=[(train_set, train_loss)], epochs=10, warmup_steps=100)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

...

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
# Multi-clustering

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import pairwise_distances_argmin_min
import numpy as np

embeddings_negatif = model.encode(df[df['sentiment'] == 'negatif']['cleaned_lemma'].tolist(), show_progress_bar=True)

scaler = StandardScaler()
embeddings_scaled = scaler.fit_transform(embeddings_negatif)

kmeans = KMeans(n_clusters=8, random_state=42)
kmeans.fit(embeddings_scaled)

distances = kmeans.transform(embeddings_scaled)

seuil = np.percentile(distances, 10)  # seuil aux 10% plus proches

associations = distances < seuil # association multiple en fonction du seuil


Batches:   0%|          | 0/868 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [24]:
# Afficher 5 commentaires pour chaque cluster pour identifier les thématiques

df_negatif = df[df['sentiment'] == 'negatif'].reset_index(drop=True)
commentaires_negatifs = df_negatif['cleaned_lemma'].tolist()

commentaires_par_cluster = {i: [] for i in range(8)}

for i, commentaire in enumerate(commentaires_negatifs):
    clusters_associes = np.where(associations[i])[0]
    for cluster in clusters_associes:
        commentaires_par_cluster[cluster].append(commentaire)

for cluster, commentaires in commentaires_par_cluster.items():
    print(f"Cluster {cluster}:")
    for commentaire in commentaires[:5]:
        print(commentaire)
    print("---------\n")

Cluster 0:
commande passer articles . article facturer envoye . quelque jour tard envoyer article article facturer fois article cher payer date livraison disparaitre message avertir etre prevenir disponible impossible annuler commande . contact reprise sav promettre remboursement article facturer fois tord rappel arriver jamais . expliquer souhait annuler article non disponible rembourser fois transmettre requete service competent jamais reponse . savoir incompetent malhonnete
commander jeu video noel mari renvoyer remboursement article recevoir demande rappeler colis arriver vide ! ! deduire quelqu servir passage demande preuve depot poids indiquer prouver envoyer enveloppe carton vide envoi photo final repondre negatif remboursement deplacer poste chercher justificatif final remboursee . encourager clairement salarie prendre colis reception savoir requete aboutir . jour commande arrivee domicile fois . beaucoup trop vol mauvais foi mieux aller discount cher passage
jamais entreprise 

In [25]:
# attitrer les clusters et afficher les thématisations

themes = {
    0: "Expériences négatives avec le Service Client",
    1: "Problème de comptes",
    2: "Problème de commandes",
    3: "Problèmes de livraison",
    4: "Échecs logistiques",
    5: "Problèmes de remboursement",
    6: "Manque de responsabilité",
    7: "Insatisfactions continues et non-résolues"
}


for i, commentaire in enumerate(df[df['sentiment'] == 'negatif']['cleaned_lemma']):
    clusters_associes = np.where(associations[i])[0]


In [28]:
# Compter le nombre de fois que les clusters sont associés à un comm

count_clusters = {cluster: 0 for cluster in themes.keys()}

for i, commentaire in enumerate(df[df['sentiment'] == 'negatif']['cleaned_lemma']):
    clusters_associes = np.where(associations[i])[0]
    for cluster in clusters_associes:
        count_clusters[cluster] += 1  # Utilisez l'entier directement sans conversion en chaîne
sorted_clusters = sorted(count_clusters.items(), key=lambda item: item[1], reverse=True)

for cluster, compte in sorted_clusters:
    print(f"Cluster {cluster} ('{themes[cluster]}'): {compte} fois")



Cluster 4 ('Échecs logistiques'): 4905 fois
Cluster 5 ('Problèmes de remboursement'): 3385 fois
Cluster 3 ('Problèmes de livraison'): 3305 fois
Cluster 2 ('Problème de commandes'): 2873 fois
Cluster 7 ('Insatisfactions continues et non-résolues'): 2227 fois
Cluster 6 ('Manque de responsabilité'): 2136 fois
Cluster 0 ('Expériences négatives avec le Service Client'): 2123 fois
Cluster 1 ('Problème de comptes'): 1244 fois


In [30]:
# Affichage des commentaires qui sont associés à au moins 3 clusters

commentaires_3clusters = []

for i, commentaire in enumerate(df[df['sentiment'] == 'negatif']['cleaned_lemma']):
    clusters_associes = np.where(associations[i])[0]
    if len(clusters_associes) >= 3:
        commentaires_3clusters.append((commentaire, clusters_associes))

donnees = [{
    "commentaire": commentaire,
    "clusters associés": ', '.join([themes[cluster] for cluster in clusters])
} for commentaire, clusters in commentaires_3clusters[:10]]

df_commentaires_3clusters = pd.DataFrame(donnees)

pd.set_option('display.max_colwidth', None)
df_commentaires_3clusters


,commentaire,clusters associés
0,livraison catastrophique avoir notifier compte amazone colis etre livrer remettre gardien livraison lieu soir gardien recuperer colis colis disparaitre retrouver rue retrouver jardin retrouver arriver service client patienter enquete lancer impossible contacter livreur heure auparavant cense presenter dramatique recevoir colis,"Problèmes de livraison, Échecs logistiques, Problèmes de remboursement, Manque de responsabilité"
1,commander fevrier recevoir commande livrer prime jour lampe casse livreur sonner descendre livreur partir abandonner colis terre rester colis commande meuble teler commode bout jour aucun livraison fois appeler chronopost amazon dire livrer lendemain obliger rester cloitre enfant vacance jour passer rien demande remboursement remboursement lundi appeler remboursement recevoir prendre semaine aboutir recevoir commode entre temps colis devoir recevoir commode casser plein impact etagere etagere casser abimer retourne meuble semaine livreur passer rembourser aujourd hui etre mars remboursement,"Problèmes de livraison, Échecs logistiques, Problèmes de remboursement"
2,commander lot cle usb livraison correctement passer recuperer boite lettre enveloppe vide preparateur omettre mettre oup lancer appel pdg mieux remunerer pouvoir payer balle cle usb bon remboursement etre rapide heureusement,"Échecs logistiques, Problèmes de remboursement, Insatisfactions continues et non-résolues"
3,payer precque balle jeu niece fin janvier commande expedier dire probleme recevoir remboursement sommer aujourd hui fevrier aucun remboursement deconseiller fortement site vole cacahuete,"Problème de commandes, Échecs logistiques, Problèmes de remboursement, Insatisfactions continues et non-résolues"
4,commande passer articles . article facturer envoye . quelque jour tard envoyer article article facturer fois article cher payer date livraison disparaitre message avertir etre prevenir disponible impossible annuler commande . contact reprise sav promettre remboursement article facturer fois tord rappel arriver jamais . expliquer souhait annuler article non disponible rembourser fois transmettre requete service competent jamais reponse . savoir incompetent malhonnete,"Expériences négatives avec le Service Client, Problème de commandes, Échecs logistiques, Problèmes de remboursement, Insatisfactions continues et non-résolues"
5,bonjour commander article corresponder desirer jai mois remboursement rembourser totalite politique remboursement totalite super decevoir relance arret rembourser aucun reponse jamais commander site,"Problèmes de livraison, Échecs logistiques, Problèmes de remboursement, Insatisfactions continues et non-résolues"
6,passer bien service client amazon probleme rentrer petit case prevoir systeme . compter service client essence incapable analyser moindre dossier multiple interlocuteur succeder tenir compte historique dossier ecouter comprendre probleme resoudre but simplement traiter demande vite passer client service client totalement desincarner plein ailleurs cœur personne amener travailler condition quotidien changer rien experience qualite service demeurer absolument deplorable,"Problème de commandes, Échecs logistiques, Manque de responsabilité"
7,commander jeu video noel mari renvoyer remboursement article recevoir demande rappeler colis arriver vide ! ! deduire quelqu servir passage demande preuve depot poids indiquer prouver envoyer enveloppe carton vide envoi photo final repondre negatif remboursement deplacer poste chercher justificatif final remboursee . encourager clairement salarie prendre colis reception savoir requete aboutir . jour commande arrivee domicile fois . beaucoup trop vol mauvais foi mieux aller discount cher passage,"Expériences négatives avec le Service Client, Problème de commandes, Échecs logistiques, Problèmes de remboursement, Insatisfactions continues et non-résolues"
8,colis egarer arreter faire livrer commande chronopost,"Problèmes de livraison, Éch

In [31]:
# Score de silhouette pour évaluer la qualité des clusters

from sklearn.metrics import silhouette_score

silhouette_avg = silhouette_score(embeddings_scaled, kmeans.labels_)
print(f'Silhouette Score: {silhouette_avg}')

Silhouette Score: 0.05404871329665184


Sans surprise, le score de silhouette est bas et indique une séparation insuffisante entre les clusters. Cela est cohérent puisque les commentaires soulèvent plusieurs thématiques. Le multiclustering tentait de pallier à cette limite en reconnaissant que certains commentaires peuvent appartenir à plusieurs thèmes. Cependant, avec le peu d'exemples affichés, il est difficile de vérifier la précision de cette méthode sur l'ensemble du dataset. De plus,la méthode utilisée pour attribuer des thèmes aux clusters est basée sur une lecture manuelle de seulement cinq commentaires par cluster, ce qui pourrait ne pas être représentatif de l'ensemble des commentaires dans chaque groupe.

Cette deuxième méthode soulève aisin des questions sur la fiabilité des associations faites entre les commentaires et les clusters.